In [ ]:
import os
from dotenv import load_dotenv
from roboflow import Roboflow
from ultralytics import YOLO

In [ ]:
#ENV_PATH = os.path.join(os.path.dirname(__file__), '../../', '.env') # This if .py file
ENV_PATH = os.path.join(os.getcwd(), '../../', '.env')                # This if .ipynb file
load_dotenv(dotenv_path=ENV_PATH)
ROBOFLOW_API_KEY = os.getenv('ROBOFLOW_API_KEY')

In [ ]:
rf = Roboflow(api_key=ROBOFLOW_API_KEY)
project = rf.workspace("roboflow-jvuqo").project("football-field-detection-f07vi")
version = project.version(15)
dataset = version.download("yolov8")

In [ ]:
os.system(f"sed -i 's|\\(train: \\).*|\\1../train/images|' {dataset.location}/data.yaml")
os.system(f"sed -i 's|\\(val: \\).*|\\1../valid/images|' {dataset.location}/data.yaml")

In [ ]:
# YOLO model
model = YOLO('yolov8x-pose.pt')
# Training configuration
model.train(
    data   = f'{dataset.location}/data.yaml',
    task   = 'pose',
    epochs = 500,                            
    imgsz  = 640,                            
    batch  = 48,                          
    mosaic = 0.0,   # to avoid mosaic augmentation                        
    device = [0,1], # if you have one single GPU, use device=0
    workers = 1
)